In [16]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200


import basedosdados as bd

from io import StringIO
import requests

In [65]:
def _sheet_to_df(columns_config_url):
    url = columns_config_url.replace("edit#gid=", "export?format=csv&gid=")
    try:
        return pd.read_csv(StringIO(requests.get(url).content.decode("utf-8")))
    except:
        raise Exception(
            "Check if your google sheet Share are: Anyone on the internet with this link can view"
        )
        
def normalize_cols(df):
    return (
        df.str.normalize("NFKD")
        .str.encode("ascii", errors="ignore")
        .str.decode("utf-8")
        .str.title()

    )

In [195]:

df = _sheet_to_df('https://docs.google.com/spreadsheets/d/1kfmQCZtRqc4msId0mtCwXbgAmubLm4SS_ReqMLm4Gz0/edit#gid=1834982883')
df = df[['ano','nome_escola','id_municipio','id_escola','municipio','matriculas']]
# df['municipio'] =  normalize_cols(df['cidade'])
df['id_escola'] = df['id_escola'].astype('str')
df['id_municipio'] = df['id_municipio'].astype('str')

In [201]:
len(df['id_municipio'].unique())

32

In [202]:
len(df['id_escola'].unique())

78

In [199]:
df.drop_duplicates('id_escola')['matriculas'].sum()

35985

In [205]:
vale = dd[['id_municipio','municipio']].drop_duplicates()

In [206]:
mask = vale['id_municipio'].isin(df['id_municipio'].tolist())

In [208]:
vale[np.logical_not(mask)]

,id_municipio,municipio
1,3520400,Ilhabela
59,3502507,Aparecida
75,3526308,Lagoinha
97,3509700,Campos do Jordão
98,3531704,Monteiro Lobato
109,3548203,Santo Antônio do Pinhal
145,3520202,Igaratá
159,3554805,Tremembé


In [180]:
# q = """

# with municipios as (
#   SELECT
#     id_municipio,
#     municipio, 
#     mesorregiao
#   FROM `basedosdados.br_bd_diretorios_brasil.municipio`
#   where sigla_uf='SP' and (mesorregiao = 'Vale do Paraíba Paulista' or id_municipio='3530607')
# ),

# escolas as (
#   SELECT
#     id_escola,
#     id_municipio,
#     nome_escola,
#     rede
#   FROM `basedosdados.br_bd_diretorios_brasil.escola`
#   where rede = 'estadual'
# ),

# escola_municipio as (
# select
#   municipios.id_municipio,
#   municipios.municipio,
#   municipios.mesorregiao,
#   escolas.id_escola,
#   escolas.nome_escola,
#   escolas.rede
# from escolas 
# JOIN municipios
# ON municipios.id_municipio = escolas.id_municipio
# ),

# alunos as (
#   SELECT
#     id_escola,
#     sum(1) as matriculas
#   FROM `basedosdados.br_inep_censo_escolar.matricula` 
#   where ano=2020 and sigla_uf='SP' and rede = 'estadual' 
#   GROUP BY id_escola
# )


# select 
#   escola_municipio.*,
#   alunos.matriculas 
# from escola_municipio
# JOIN alunos
# ON escola_municipio.id_escola = alunos.id_escola 

# """
# dd = bd.read_sql(q, billing_project_id='basedosdados-dev')
# dd = dd.drop_duplicates()




In [185]:
len(dd['id_escola'].unique())

358

In [187]:
escolas_matriculas = df.merge(dd, how = 'left', on=['id_municipio','id_escola'])

In [189]:
escolas_matriculas.head()

,ano,escola,id_municipio,id_escola,cidade,municipio,mesorregiao,nome_escola,rede,matriculas
0,2020.0,VICENTE DE PAULA ALMEIDA,3503158,35012488,ARAPEI,Arapeí,Vale do Paraíba Paulista,VICENTE DE PAULA ALMEIDA,estadual,98.0
1,2020.0,BARAO DA BOCAINA,3503505,35012543,AREIAS,Areias,Vale do Paraíba Paulista,BARAO DA BOCAINA,estadual,136.0
2,2020.0,SAO LAURINDO VISCONDE,3504909,35012464,BANANAL,Bananal,Vale do Paraíba Paulista,SAO LAURINDO VISCONDE,estadual,322.0
3,2020.0,PEREIRA DE MATTOS DOUTOR,3508504,35049219,CAÇAPAVA,Caçapava,Vale do Paraíba Paulista,PEREIRA DE MATTOS DOUTOR,estadual,371.0
4,2020.0,JUCA PADRE,3508603,35013080,CACHOEIRA PAULISTA,Cachoeira Paulista,Vale do Paraíba Paulista,JUCA PADRE,estadual,332.0


In [190]:
escolas_matriculas[['municipio', 'id_escola', 'nome_escola', 'matriculas']]

,municipio,id_escola,nome_escola,matriculas
0,Arapeí,35012488,VICENTE DE PAULA ALMEIDA,98.0
1,Areias,35012543,BARAO DA BOCAINA,136.0
2,Bananal,35012464,SAO LAURINDO VISCONDE,322.0
3,Caçapava,35049219,PEREIRA DE MATTOS DOUTOR,371.0
4,Cachoeira Paulista,35013080,JUCA PADRE,332.0
5,Canas,35013134,ALICE VILELA GALVAO PROFA,238.0
6,Caraguatatuba,35011290,EDUARDO CORREA DA COSTA JUNIOR DOUTOR,263.0
7,Cruzeiro,35012506,OSWALDO CRUZ,534.0
8,Cunha,35012786,GERALDO COSTA,308.0
9,Jacareí,35013973,DIRCEU JUNQUEIRA DE SOUZA PROF,516.0


415

In [183]:
escola = 'JUCA'

mask = (dd['nome_escola'].str.contains(escola))##& (dd['id_municipio']==id_municipio)
dd[mask].drop('id_municipio',1)

,municipio,mesorregiao,id_escola,nome_escola,rede,matriculas
84,Cachoeira Paulista,Vale do Paraíba Paulista,35013080,JUCA PADRE,estadual,332


In [172]:
"""
with municipios as (
  SELECT
    id_municipio,
    municipio, 
    mesorregiao
  FROM `basedosdados.br_bd_diretorios_brasil.municipio`
  where sigla_uf='SP' and (mesorregiao = 'Vale do Paraíba Paulista' or id_municipio='3530607')
),

escolas as (
  SELECT
    id_escola,
    id_municipio,
    nome_escola,
    rede
  FROM `basedosdados.br_bd_diretorios_brasil.escola`
  where rede = 'estadual'
),

alunos as (
  SELECT
    id_escola,
    rede
  FROM `basedosdados.br_inep_censo_escolar.escola` id_escola
  where rede = 'estadual'
)

select
  municipios.id_municipio,
  municipios.municipio,
  municipios.mesorregiao,
  escolas.id_escola,
  escolas.nome_escola,
  escolas.rede
from escolas 
JOIN municipios
ON municipios.id_municipio = escolas.id_municipio

"""

"\nwith municipios as (\n  SELECT\n    id_municipio,\n    municipio, \n    mesorregiao\n  FROM `basedosdados.br_bd_diretorios_brasil.municipio`\n  where sigla_uf='SP' and (mesorregiao = 'Vale do Paraíba Paulista' or id_municipio='3530607')\n),\n\nescolas as (\n  SELECT\n    id_escola,\n    id_municipio,\n    nome_escola,\n    rede\n  FROM `basedosdados.br_bd_diretorios_brasil.escola`\n  where rede = 'estadual'\n),\n\nalunos as (\n  SELECT\n    id_escola,\n    rede\n  FROM `basedosdados.br_inep_censo_escolar.escola` id_escola\n  where rede = 'estadual'\n)\n\nselect\n  municipios.id_municipio,\n  municipios.municipio,\n  municipios.mesorregiao,\n  escolas.id_escola,\n  escolas.nome_escola,\n  escolas.rede\nfrom escolas \nJOIN municipios\nON municipios.id_municipio = escolas.id_municipio\n\n"

In [41]:
dd['municipio'] = normalize_cols(dd['municipio'])
df['municipio'] = normalize_cols(df['cidade'])


,ano,escola,cidade,municipio
0,2020.0,VICENTE DE PAULA ALMEIDA,ARAPEI,Arapei
1,2020.0,BARAO DA BOCAINA,AREIAS,Areias
2,2020.0,SAO LAURINDO VISCONDE,BANANAL,Bananal
3,2020.0,PEREIRA DE MATTOS DOUTOR,CAÇAPAVA,Cacapava
4,2020.0,JUCA PADRE,CACHOEIRA PAULISTA,Cachoeira Paulista
5,2020.0,ALICE VILELA GALVAO PROFA,CANAS,Canas
6,2020.0,EDUARDO CORREA DA COSTA JUNIOR DOUTOR,CARAGUATATUBA,Caraguatatuba
7,2020.0,OSWALDO CRUZ,CRUZEIRO,Cruzeiro
8,2020.0,GERALDO COSTA,CUNHA,Cunha
9,2020.0,DIRCEU JUNQUEIRA DE SOUZA,JACAREÍ,Jacarei


,ano,escola,cidade,diretor,contato,visita,pagamento,fiscalizador,endereco,finalidade,valor,enviado_plano_aplicacao,enviado_secretaria_educacao,recebeu_recurso,nse,expectativas,desvios,prioridades,educacao_empreendedora,visita_agendada,grupo_whatsapp,observacao,Unnamed: 22,municipio
0,2020.0,VICENTE DE PAULA ALMEIDA,ARAPEI,Jonas (vice-diretor),(12)3115-1110 | (12)99781-0192,reunião online 23/3,PAGO EM 11/12,NaN,NaN,CUSTEIO,"20,000.00",SIM,SIM,SIM,0.00,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,Arapei
1,2020.0,BARAO DA BOCAINA,AREIAS,Denise (Diretora),(12)99656-5575 | (12)3107-1344,NaN,PAGO EM 20/11,NaN,NaN,CUSTEIO,"20,000.00",SIM,NaN,SIM,0.00,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,Areias
2,2020.0,SAO LAURINDO VISCONDE,BANANAL,Rosângela,(12)99778-3010,Reunião online 23/6,PAGO EM 18/12,NaN,NaN,CUSTEIO,"20,000.00",SIM,SIM,sim,0.00,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,Bananal
3,2020.0,PEREIRA DE MATTOS DOUTOR,CAÇAPAVA,Helimara,(12)99161-1408,NaN,pago em 27/01,NaN,NaN,CAPITAL,"25,202.00",SIM,SIM,NaN,53.78,2.652134,1.530000,AQUISIÇÃO DE MATERIAIS DE ROBÓTICA,AGUARDANDO ENVIAR TERMO,NaN,NaN,NaN,NaN,Cacapava
4,2020.0,JUCA PADRE,CACHOEIRA PAULISTA,Diretor Jurandir,(12)99758-1326,recusou participar,PAGO EM 13/11,Rafael Diretor Jurandir,NaN,CUSTEIO,"20,000.00",SIM,SIM,SIM,50.92,2.436776,0.850000,NaN,NaN,NaN,NaN,NaN,NaN,Cachoeira Paulista
5,2020.0,ALICE VILELA GALVAO PROFA,CANAS,Rosana diretora Regina Ferreira,(12)3151-1556 | 3151-1131 | (12)99250-0629 | (12)99777-6659,reunião online 14/4,PAGO EM 11/12,Rosana diretora,NaN,CUSTEIO,"20,000.00",SIM,SIM,SIM,0.00,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,Canas
6,2020.0,EDUARDO CORREA DA COSTA JUNIOR DOUTOR,CARAGUATATUBA,Adriano diretor,(12)99127-6980,Visita realizada maio,PAGO EM 11/12,Ayslan Adriano diretor,NaN,CUSTEIO,"20,000.00",SIM,SIM,SIM,50.67,2.417951,1.250000,reforma quadra de esportes,Verificar se recebeu o termo,NaN,NaN,NaN,NaN,Caraguatatuba
7,2020.0,OSWALDO CRUZ,CRUZEIRO,Thiago,(12)99204-5419,reunião online 14/4,PAGO EM 23/12,Prof. Paula Saulo Carlos,NaN,CUSTEIO,"20,000.00",SIM,SIM,NaN,NaN,NaN,NaN,Custeio para compra de material permanente (robótica),NaN,NaN,NaN,NaN,NaN,Cruzeiro
8,2020.0,GERALDO COSTA,CUNHA,Angela Mota,(12)99751-2177,reunião online 23/3,PAGO EM 20/11,Mateus Mario Neto,NaN,CUSTEIO,"20,000.00",SIM,SIM,SIM,47.53,2.181509,0.860000,Manutenção rede elétrica e hidraulica \n,Verificar se recebeu o termo,NaN,NaN,NaN,NaN,Cunha
9,2020.0,DIRCEU JUNQUEIRA DE SOUZA,JACAREÍ,Edina,(12)99196-6961,NaN,NaN,Marcos Colozovski Cida (diretora),NaN,CAPITAL,"25,000.00",SIM,SIM,NaN,50.76,2.420000,-0.090000,"problemas estruturais \nReforma elétrica,\nReforma hidráulica,\nReforma banheiros,\nPintura Quadra,\nPortão principal precisa ser substituído,",NaN,NaN,NaN,NaN,NaN,Jacarei
